# Series 04. Part III - Deploying Pytorch Model to Sagemaker

Welcome to the series. This is the second part of tutorial series #04. The overview of tutorial series #04 is as follows:

- **Part I  :** Convolutional Neural Network  [[notebook](#04.a.Convolutional_Neural_Net)]
- **Part II :** Hands-on with Kannada MNIST dataset using Pytorch [[notebook](#04.b.Kaggle_Kannada_Pytorch)]
- **Part III:** Pytorch model training and deployment in AWS Sagemaker (this notebook)

---

## Getting Started


### Background

Kannada is a language spoken predominantly by people of Karnataka in southwestern India. The language has roughly 45 million native speakers and is written using the Kannada script. [Wikipedia](https://en.wikipedia.org/wiki/Kannada)


![Kannada](images/kannada_banner.png)

The challenge faced by many industry and banking instances in India is to build a Optical Character Recognition that can detect Kannada digits from handwriting in cheques, written documents, etc to automate important task. By building a simple digit recognition, we can help many bussiness which application relies on reading handwritten digits, e.g. automating thousands of transaction from bank cheque validation.

A study in the paper of: Prabhu, Vinay Uday nicely captured the Kannada handwritten digits and the author has kindly shared with us the dataset we can experiment with. "Kannada-MNIST: A new handwritten digits dataset for the Kannada language." arXiv preprint [arXiv:1908.01242 (2019)](https://arxiv.org/abs/1908.01242).

### Goal
This tutorial will show you how to deploy a Kannada CNN that we have built in the previous tutorial [here](#04.b.Kaggle_Kannada_Pytorch). The dataset we are using is the [Kannada MNIST dataset](https://www.kaggle.com/c/Kannada-MNIST), a playground dataset from Kaggle. Here you can expect to get an insight about:
- Understand modelling data science challenge using Pytorch
- The challenge and solution for deploying model for use ready in cloud
- The integrated deployment flow with Pytorch and AWS Sagemaker

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.


## Setup 
_(Duration: 5 min)_

_This notebook was created and tested on an ml.m4.xlarge notebook instance_ with *conda_pytorch_p36 environment*.

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

---

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/kannada-processed'

role = sagemaker.get_execution_role()

### Packages

In [ ]:
# standard library
import os, sys
import math
import json, logging, argparse

In [ ]:
# data analytics library
import numpy as np
import pandas as pd

In [ ]:
# Pytorch Library
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.optim as optim
import torch.nn.functional as F

# Pytorch Utilities and Vision Library
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler

from torchvision import datasets, transforms

In [ ]:
# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Data

### Getting the data
Continuing from the previous tutorial, we are going to use [Kannada-MNIST](#) dataset from Kaggle that we have downloaded previously. 

> **_NOTE:_**  If you skipped the previous tutorial, make sure you go through section [__Downloading Kaggle Dataset__](#04.b.Kaggle_Kannada_Pytorch) in part II of our notebook series #04. Then afterward copy the processed data to your current environment at `data/Kannada/processed` directory.

In [ ]:
data_dir = 'data/Kannada/processed/'

In [ ]:
# Check the data folder content
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print('data at: ' + os.path.join(dirname, filename))

### Loading Dataset

#### Dataset Files

There are 4 files we get after downloading the data from Kaggle. According to the official docs above, the files summary is as follows:
- `train.csv` - the training set. The first column of every row is the label, the rest 784 values in the same row is the pixel value of our image with flattened structure defined above.
- `Dig-MNIST.csv`: this is similar to image training set in `train.csv`. The contributor provide this set kindly to allow us to have a validation/testing set before we make a submission.
- `test.csv`: the submission set. Unlike the `train.csv` and `Dig-MNIST.csv` it doesn't come with label because it suppose to be our submission for the competition. We will refer data from this file as *submission set* from now on.
- `sample_submission.csv` - a sample submission file in the correct format

Loading data may take a few moments.

In [ ]:
from sklearn.model_selection import train_test_split

# # Load Data
train_images = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None).values
val_images = pd.read_csv(os.path.join(data_dir, 'validation.csv'), header=None).values
test_images = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values

# Load Labels, must be a list of indices (int)
train_labels = pd.read_csv(os.path.join(data_dir, 'train_labels.csv'),
                           header=None).values.squeeze().tolist()
val_labels = pd.read_csv(os.path.join(data_dir, 'validation_labels.csv'),
                         header=None).values.squeeze().tolist()
test_labels = pd.read_csv(os.path.join(data_dir, 'test_labels.csv'),
                          header=None).values.squeeze().tolist()

### Uploading the data to S3
Next, we are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

In [ ]:
s3_data_path = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('Data path/url in AWS S3 -> {}'.format(s3_data_path))

## Train Session in Cloud
*(Duration: 20 min)*

### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For the script run by this notebook:

In [ ]:
!pygmentize kannada_entrypoint.py

### Pytorch Distributed Computing in Sagemaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `kannada_entrypoint.py` script above.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='kannada_entrypoint.py',    # script for training entrypoint
                    dependencies=['pytorch_utils'],   # source code for model and other functionals
                    role=role,
                    framework_version='1.2.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 2,
                        'batch_size': 128,
                        'lr': 4e-3,
                        'num_classes': 10,
                        'backend': 'gloo'
                    })

In [ ]:
estimator.fit({'training': s3_data_path})

## Host
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Evaluate
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
import numpy as np

image = np.array(test_images[0], dtype=np.uint8) / 255
image = image.reshape(1, 1, 28, 28)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
estimator.delete_endpoint()